In [ ]:
import os
import subprocess
import time
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    Reuses saved credentials to avoid repeated logins.
    """
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    return build("youtube", "v3", credentials=creds)

def get_video_duration(video_path):
    """
    Returns the duration of a video in seconds.
    """
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries", "format=duration",
             "-of", "default=noprint_wrappers=1:nokey=1", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        return float(result.stdout.strip())
    except Exception as e:
        print(f"⚠️ Warning: Could not read duration for {video_path}. Error: {e}")
        return None

def try_merge_videos(folder_path, video_files, expected_duration, merge_attempt):
    """
    Attempts to merge videos directly without conversion. If the merged duration 
    is incorrect, fall back to converting and merging again.
    """
    file_list_path = os.path.join(folder_path, "file_list.txt")
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")

    with open(file_list_path, "w") as f:
        for video_file in video_files:
            f.write(f"file '{video_file}'\n")

    print(f"🔗 Attempt {merge_attempt}: Merging videos...")
    subprocess.run(
        ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list_path, "-c", "copy", merged_video_path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    merged_duration = get_video_duration(merged_video_path)
    if merged_duration and abs(merged_duration - expected_duration) <= 5:
        print(f"✅ Merge successful! Duration: {merged_duration:.2f} sec")
        return merged_video_path
    else:
        print(f"❌ Merge failed. Merged duration: {merged_duration:.2f} sec, Expected: {expected_duration:.2f} sec")
        os.remove(merged_video_path)  # Delete incorrect merge
        return None  # Signal to perform full conversion

def merge_videos_and_create_timestamps(folder_path):
    """
    Converts MP4 videos to a uniform format if needed, merges them, and creates timestamps.
    Returns the merged video path, description content (timestamps), and total duration.
    """
    print(f"📂 Checking folder: {folder_path}")

    if not os.path.exists(folder_path):
        print(f"❌ Error: The folder path '{folder_path}' does not exist.")
        return None, None, None

    video_files = []
    timestamps = []
    total_duration = 0

    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(".mp4"):
            filepath = os.path.join(folder_path, filename)
            video_files.append(filepath)
            duration = get_video_duration(filepath)
            if duration:
                timestamps.append((filename, total_duration))
                total_duration += duration

    if not video_files:
        print("❌ No MP4 files found.")
        return None, None, None

    # ✅ Try merging original videos first
    merged_video_path = try_merge_videos(folder_path, video_files, total_duration, merge_attempt=1)
    if merged_video_path:
        return merged_video_path, "\n".join(f"{f} - {t}" for f, t in timestamps), total_duration

    # ✅ Convert videos and merge again if direct merge failed
    converted_folder = os.path.join(folder_path, "converted_videos")
    os.makedirs(converted_folder, exist_ok=True)
    converted_files = []

    print("🔄 Converting videos due to merge failure...")
    for idx, video in enumerate(video_files):
        converted_video = os.path.join(converted_folder, f"converted_{idx}.mp4")
        print(f"🔄 Converting: {os.path.basename(video)} ...")
        subprocess.run(
            ["ffmpeg", "-i", video, "-c:v", "libx264", "-c:a", "aac", "-strict", "experimental", converted_video],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        converted_files.append(converted_video)
        print(f"✅ Finished converting: {os.path.basename(video)}")

    # ✅ Merge converted videos
    return try_merge_videos(folder_path, converted_files, total_duration, merge_attempt=2), "\n".join(f"{f} - {t}" for f, t in timestamps), total_duration

def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube link.txt'.
    """
    print(f"📤 Uploading: {title}...")
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description,
                "tags": ["training", "tutorial", "video"],
                "categoryId": "27"
            },
            "status": {
                "privacyStatus": "unlisted"
            }
        },
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploaded {int(status.progress() * 100)}%")
        except Exception as e:
            raise Exception(f"Error during upload: {e}")

    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube link.txt"), "w") as f:
        f.write(video_url + "\n")

    print(f"✅ Upload complete! Video URL: {video_url}")
    return video_url

def cleanup_files(folder_path, total_duration):
    """
    Deletes original and converted videos if the merged video duration is within ±5 seconds.
    """
    merged_video_path = os.path.join(folder_path, "merged_video.mp4")
    merged_duration = get_video_duration(merged_video_path)

    if merged_duration and abs(merged_duration - total_duration) <= 5:
        print("🧹 Cleanup: Deleting original and converted videos...")

        for filename in os.listdir(folder_path):
            if filename.lower().endswith(".mp4"):
                os.remove(os.path.join(folder_path, filename))

        converted_folder = os.path.join(folder_path, "converted_videos")
        if os.path.exists(converted_folder):
            for filename in os.listdir(converted_folder):
                os.remove(os.path.join(converted_folder, filename))
            os.rmdir(converted_folder)

        print("✅ Cleanup completed!")
    else:
        print("⚠️ Cleanup skipped: Merged video duration mismatch!")

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path) and any(f.lower().endswith('.mp4') for f in os.listdir(subfolder_path)):
            print(f"🚀 Processing folder: {subfolder_path}")

            merged_video_path, description, total_duration = merge_videos_and_create_timestamps(subfolder_path)

            if merged_video_path and description:
                folder_parts = os.path.normpath(subfolder_path).split(os.sep)
                title = " - ".join(folder_parts[-4:]) if len(folder_parts) >= 4 else "Merged Video"

                upload_to_youtube(merged_video_path, title, description, subfolder_path)

                cleanup_files(subfolder_path, total_duration)

    print("✅ All subfolders processed!")
